# Starting Lyrics Processing

###### Importing all libraries needed

In [1]:
import numpy as np

import nltk

from sklearn.preprocessing import normalize
from sklearn import preprocessing
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import ShuffleSplit
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
import copy
from scipy.stats import friedmanchisquare as friedman
from scipy.stats import wilcoxon as wilcoxon
from scipy.stats import ttest_ind as ttest
from scipy.stats import ttest_ind_from_stats as ttest2
from scipy.stats import f_oneway as f_oneway
from matplotlib import pyplot as plt
import csv
import os
from bs4 import BeautifulSoup
import codecs
from tqdm import tqdm_notebook, tnrange

nltk.download('punkt')

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


[nltk_data] Downloading package punkt to
[nltk_data]     /home/felipetanios/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

###### getting all the lyrics

In [2]:
tracks_names = []
tracks_lyrics = []

for root, dirs, files in os.walk("."):
    for f in files:
        if f[-3:] == "txt":
            tracks_names.append(f)
            tracks_path = os.path.join(root, f)
            track_single_lyric =  open (tracks_path, 'r')
            tracks_lyrics.append(track_single_lyric.read().lower())


for i in range(len(tracks_names)):
    tracks_names[i] = tracks_names[i][:-4]

print (tracks_lyrics)
print (tracks_names)
print (len(tracks_names))


[' madalena o meu peito percebeu que o mar é uma gota comparado ao pranto meu.  fique certa quando o nosso amor desperta logo o sol se desespera e se esconde lá na serra.  madalena o que é meu não se divide nem tão pouco se admite quem do nosso amor duvide.  até a lua se arrisca num palpite que o nosso amor existe forte ou fraco alegre ou triste.  ', ' depois de trabalhar toda a semana meu sábado não vou desperdiçar já fiz o meu programa pra esta noite e sei por onde começar  um bom lugar para encontrar: copacabana prá passear à beira-mar: copacabana depois num bar à meia-luz: copacabana eu esperei por essa noite uma semana  um bom jantar depois de dançar: copacabana um só lugar para se amar: copacabana a noite passa tão depressa, mas vou voltar se pra semana eu encontrar um novo amor: copacabana  ', ' pai, afasta de mim esse cálice pai, afasta de mim esse cálice pai, afasta de mim esse cálice de vinho tinto de sangue  pai, afasta de mim esse cálice pai, afasta de mim esse cálice pai, 

<h6>creating dictionaries associating lyrics names and their percieved emotion</h6>

In [3]:
dataset = {}
with open ('base.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        dataset[row[0][:-4]] = row[2]
    
print (dataset)

{'Elis_Regina_-_Madalena': 'Happy', 'Elis_Regina_-_Como_Nossos_Pais': 'Sad', 'Samba_de_Uma_Nota_So': 'Happy', 'Tom_Jobim_-_Luiza': 'Sad', 'Onde_anda_voce_Toquinho_Vinicius_de_moraes': 'Sad', 'Sergio_Mendes_+_Brasil_66_-_Mas_Que_Nada': 'Happy', 'Tarde_em_Itapoa_-_Vinicius_e_Toquinho': 'Happy', 'Chico_Buarque_de_Hollanda_-_Geni_e_o_Zepelim_(1979)': 'Sad', 'Sabado_em_Copacabana': 'Happy', 'Marcos_Valle_-_Samba_de_verao': 'Happy', 'Soneto_de_Separacao_-_Elis_e_Tom': 'Sad', 'Antonio_Carlos_Jobim_-_So_Danco_Samba': 'Happy', 'Gal_Costa_-_Anos_Dourados': 'Sad', 'Joao_Gilberto_-_Ela_e_Carioca': 'Happy', 'Elis_Regina_-_Joao_e_Maria': 'Sad', 'Elis_Regina_-_Carinhoso_(Meu_Coracao)': 'Sad', 'Gilberto_Gil_-_Rosa_Morena': 'Happy', 'eu_nao_existo_sem_voce_-_Maria_Bethania': 'Sad', 'Milton_Nascimento_-_Travessia': 'Sad', 'Jorge_Ben_Jor_-_Que_Maravilha': 'Happy', 'Elis_Regina_-_O_Bebado_e_a_Equilibrista': 'Sad', 'Elis_Regina_-_Maria_Maria': 'Sad', 'Minha_Namorada_-_Miucha_e_Vinicius_de_Moraes': 'Happy',

<h6>Stemming Words</h6>

<h6>Creating dictionary with all the words</h6>

In [10]:
word_dic = {}

for i in range(len(tracks_lyrics)):
    for word in tracks_lyrics[i].split():
        if word not in word_dic:
            word_dic[word] = 1
        else:
            word_dic[word] = 1 + word_dic[word]
            
print (word_dic)

{'virar': 2, 'qualquer': 9, 'desfez': 1, 'laralara...': 1, 'amor:': 1, 'misturando': 1, 'bocas': 1, 'dancei': 1, 'saudade': 2, 'lento': 1, 'canta,': 1, 'matar': 4, 'chama': 2, 'todo': 6, 'invenção': 1, 'infinito': 1, 'sinal': 1, 'enorme': 1, 'projeto': 1, 'firmamento': 1, 'merece': 1, 'desespera': 1, 'pedra,': 5, 'seguindo': 3, 'pressentimento': 1, 'pro': 4, 'porque': 6, 'queira': 2, 'no': 44, 'afobada': 1, 'influência': 3, 'engolir': 1, 'comparado': 1, 'dó.': 1, 'peroba': 1, 'amigo': 1, 'chover': 1, 'anos': 1, 'dormir': 2, 'março,': 1, 'pessoal': 4, 'sentir': 4, 'mesma': 2, 'promete': 1, 'maria"': 1, 'dominou': 1, 'conselhos': 1, 'coisa': 7, 'lembrou': 1, 'ideia!"': 1, 'pedir,': 1, 'raiou': 1, 'jazz': 4, 'apenas': 2, 'sonhou,': 1, 'assim': 15, '"te': 1, 'pode': 6, 'cores': 1, 'peixe,': 1, 'tenho': 7, 'consumado': 1, 'dizer.': 1, 'dá-me': 1, 'cá,': 1, 'grande': 6, 'onde': 11, 'nenhuma,': 1, 'cristo': 2, 'cadê': 1, 'internato': 1, 'aquela': 2, 'torto': 3, 'num': 6, 'pedra': 6, 'maiô': 1

<h6>Making Tfidf for each lyric</h6>

In [11]:
tracks_tfidf = []
for i in range(len(tracks_lyrics)):

SyntaxError: unexpected EOF while parsing (<ipython-input-11-7581a505c7d5>, line 2)